In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix, mean_squared_error, log_loss, accuracy_score


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
pd.__version__

'0.24.2'

In [3]:
df_fake = pd.read_csv('TenantInfo-and-usage_shuffled_inf.csv', nrows=10)

In [4]:
df_fake.shape

(10, 228)

In [5]:
df_fake.columns.tolist()

['TenantId',
 'CreatedDate',
 'CreateDateOfFirstSubscription',
 'CountryCode',
 'AllupSeats',
 'EXOSubscriptionsCount',
 'OD4BSubscriptionsCount',
 'SfBSubscriptionsCount',
 'TeamsSubscriptionsCount',
 'PaidCount',
 'ProjectSubscriptionsCount',
 'SPOSubscriptionsCount',
 'ActivatedSubscriptionTotalCount',
 'VisioSubscriptionsCount',
 'TrialSubscriptionsCount',
 'NonTrialSubscriptionsCount',
 'Languange',
 'DataCenterInstance',
 'DataCenterModel',
 'HasEXO',
 'HasSPO',
 'HasOD4B',
 'HasSfB',
 'HasYammer',
 'HasTeams',
 'HasTeamsFreemium',
 'HasKaizala',
 'HasProPlus',
 'HasAADP',
 'HasAIP',
 'HasAATP',
 'HasIntune',
 'HasMCAS',
 'HasWDATP',
 'HasAudioConference',
 'HasPhoneSystem',
 'HasEdiscovery',
 'HasCompliance',
 'HasThreatIntelligence',
 'HasCustomerLockbox',
 'HasOATP',
 'HasAADPP2',
 'HasAIPP2',
 'HasWindows',
 'HasO365CAS',
 'HasCASDiscovery',
 'HasPAM',
 'HasPowerBI',
 'HasPowerBIPremium',
 'HasPowerBIPro',
 'HasVisio',
 'HasProject',
 'HasNonTrial',
 'HasSubscription_AllCount

In [15]:

df = pd.read_csv('TenantInfo-and-usage_shuffled_inf.csv'#, nrows=200000
                )

In [16]:
print('full data set size {}'.format(df.shape))


full data set size (1865326, 228)


# Preprocess data

1. encode categorical data
2. transfer datetime data
3. convert nan to zero, inf to one
4. encode boolean type data
5. split dataset
6. normalize data

In [17]:
df['Train'] = df['Age'] > 360

In [18]:
def preprocess_data(df):
    print('Starting the preprocessing.')
    df_bool = df.select_dtypes(include='bool')
    df_num = df.select_dtypes(include=['float','int'])
    df_tenantId, df_cat, df_datetime = process_object_cols(df)
    
    print('Starting the encoding part.')
    df_cat = encoder_cat(df_cat)
    df_datetime = encoder_datetime(df_datetime)
    df_num.replace([np.nan, np.inf], [0, 1], inplace=True)
    df_bool = df_bool.astype(int)
    
    print('Starting to concat different parts into one dataframe')
    data = pd.concat([df_cat, df_num, df_bool, df_datetime], axis=1)
       
    return data, df_tenantId

In [19]:
def process_object_cols(df):
    cols_datetime = ['CreatedDate', 'CreateDateOfFirstSubscription','FirstPaidEXOStartDate',
       'FirstPaidSPOStartDate', 'FirstPaidOD4BStartDate',
       'FirstPaidSfBStartDate', #'FirstPaidYammerStartDate',
       'FirstPaidTeamsStartDate', 'FirstPaidProPlusStartDate',
       #'FirstPaidAADPStartDate', 'FirstPaidAIPStartDate',
       #'FirstPaidAATPStartDate', 'FirstPaidIntuneStartDate',
       #'FirstPaidMCASStartDate', 'FirstPaidO365E5SkuStartDate',
       #'FirstPaidM365E5SkuStartDate', 'FirstPaidEMSE5SkuStartDate'
                    ]
    df_datetime = df.loc[:, cols_datetime]
    
    cols_cat = ['CountryCode', 'Languange', #'DataCenterInstance', 'DataCenterModel',
       'SignupLocationInfo_Country', #'SignupLocationInfo_CountryCode',
       #'SignupLocationInfo_Region', 'TopParents_AreaName',
       'TopParents_CountryCode', #'TopParents_BigAreaName', 
       'TopParents_Industry', #'TopParents_RegionName',
       'TopParents_SegmentGroup', #'TopParents_SubRegionName',
       'TopParents_VerticalName']
    df_cat = df.loc[:, cols_cat]
    
    df_tenantid = df.loc[:,'TenantId']
    
    return df_tenantid, df_cat, df_datetime

In [20]:
# encode datetime columns
def encoder_datetime(df):
    cols = df.columns
    for i in cols:
        df[i] = pd.to_datetime(df[i], utc=True, errors='coerce').astype(int,errors='ignore')
    return df

In [21]:
# encode categorical columns
def encoder_cat(df):
    cols = df.columns
    for i in cols:
        df = pd.concat([df,pd.get_dummies(df[i],prefix=[i], dummy_na=True)],axis=1)
        df.drop([i],axis=1, inplace=True)
    return df

In [22]:
data, TenantId = preprocess_data(df)

Starting the preprocessing.
Starting the encoding part.
Starting to concat different parts into one dataframe


In [23]:
data.shape

(1865326, 1093)

In [24]:
data.to_csv('encoded_data.csv', index=False)

In [25]:
TenantId.to_csv('encoded_data_tenantId.csv', index=False, header=None)

/data/home/t-chepan/env/newlab/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [26]:
df_train = data.loc[data['Train'] == 1]
df_test = data.loc[data['Train'] == 0]

In [27]:
data.shape

(1865326, 1093)

In [28]:
df_train.shape

(1384149, 1093)

In [18]:
df_test.shape

(481177, 1088)

In [19]:
481177 / 2

240588.5

In [20]:
ytrain_exo = df_train['AR_exchange_06']
Xtrain_exo = df_train.drop(columns=['Train', 'AR_exchange_06'])

In [21]:
Xtrain_exo = Xtrain_exo.to_numpy()
ytrain_exo = ytrain_exo.to_numpy()

In [22]:
dev_size = int(df_test.shape[0] / 2)

In [23]:
dev = df_test.iloc[:dev_size, :]
test = df_test.iloc[dev_size:, :]

In [24]:
ydev_exo = dev['AR_exchange_06']
Xdev_exo = dev.drop(columns=['Train', 'AR_exchange_06'])

ytest_exo = test['AR_exchange_06']
Xtest_exo = test.drop(columns=['Train', 'AR_exchange_06'])

In [25]:
Xdev_exo = Xdev_exo.to_numpy()
ydev_exo = ydev_exo.to_numpy()

Xtest_exo = Xtest_exo.to_numpy()
ytest_exo = ytest_exo.to_numpy()

In [27]:
data.head()

,['CountryCode']_AD,['CountryCode']_AE,['CountryCode']_AF,['CountryCode']_AG,['CountryCode']_AI,['CountryCode']_AL,['CountryCode']_AM,['CountryCode']_AO,['CountryCode']_AR,['CountryCode']_AS,['CountryCode']_AT,['CountryCode']_AU,['CountryCode']_AW,['CountryCode']_AX,['CountryCode']_AZ,['CountryCode']_BA,['CountryCode']_BB,['CountryCode']_BD,['CountryCode']_BE,['CountryCode']_BF,['CountryCode']_BG,['CountryCode']_BH,['CountryCode']_BI,['CountryCode']_BJ,['CountryCode']_BL,['CountryCode']_BM,['CountryCode']_BN,['CountryCode']_BO,['CountryCode']_BQ,['CountryCode']_BR,['CountryCode']_BS,['CountryCode']_BT,['CountryCode']_BW,['CountryCode']_BY,['CountryCode']_BZ,['CountryCode']_CA,['CountryCode']_CD,['CountryCode']_CG,['CountryCode']_CH,['CountryCode']_CI,['CountryCode']_CK,['CountryCode']_CL,['CountryCode']_CM,['CountryCode']_CN,['CountryCode']_CO,['CountryCode']_CR,['CountryCode']_CV,['CountryCode']_CW,['CountryCode']_CY,['CountryCode']_CZ,['CountryCode']_DE,['CountryCode']_DJ,['CountryCode']_DK,['CountryCode']_DM,['CountryCode']_DO,['CountryCode']_DZ,['CountryCode']_EC,['CountryCode']_EE,['CountryCode']_EG,['CountryCode']_ES,['CountryCode']_ET,['CountryCode']_FI,['CountryCode']_FJ,['CountryCode']_FK,['CountryCode']_FO,['CountryCode']_FR,['CountryCode']_GA,['CountryCode']_GB,['CountryCode']_GD,['CountryCode']_GE,['CountryCode']_GF,['CountryCode']_GG,['CountryCode']_GH,['CountryCode']_GI,['CountryCode']_GL,['CountryCode']_GM,['CountryCode']_GN,['CountryCode']_GP,['CountryCode']_GQ,['CountryCode']_GR,['CountryCode']_GT,['CountryCode']_GW,['CountryCode']_GY,['CountryCode']_HK,['CountryCode']_HN,['CountryCode']_HR,['CountryCode']_HT,['CountryCode']_HU,['CountryCode']_ID,['CountryCode']_IE,['CountryCode']_IL,['CountryCode']_IM,['CountryCode']_IN,['CountryCode']_IO,['CountryCode']_IQ,['CountryCode']_IS,['CountryCode']_IT,['CountryCode']_JE,['CountryCode']_JM,['CountryCode']_JO,['CountryCode']_JP,['CountryCode']_KE,['CountryCode']_KG,['CountryCode']_KH,['CountryCode']_KN,['CountryCode']_KR,['CountryCode']_KW,['CountryCode']_KY,['CountryCode']_KZ,['CountryCode']_LA,['CountryCode']_LB,['CountryCode']_LC,['CountryCode']_LI,['CountryCode']_LK,['CountryCode']_LR,['CountryCode']_LS,['CountryCode']_LT,['CountryCode']_LU,['CountryCode']_LV,['CountryCode']_LY,['CountryCode']_MA,['CountryCode']_MC,['CountryCode']_MD,['CountryCode']_ME,['CountryCode']_MF,['CountryCode']_MG,['CountryCode']_MH,['CountryCode']_MK,['CountryCode']_ML,['CountryCode']_MM,['CountryCode']_MN,['CountryCode']_MO,['CountryCode']_MP,['CountryCode']_MQ,['CountryCode']_MR,['CountryCode']_MT,['CountryCode']_MU,['CountryCode']_MV,['CountryCode']_MW,['CountryCode']_MX,['CountryCode']_MY,['CountryCode']_MZ,['CountryCode']_NC,['CountryCode']_NE,['CountryCode']_NF,['CountryCode']_NG,['CountryCode']_NI,['CountryCode']_NL,['CountryCode']_NO,['CountryCode']_NP,['CountryCode']_NR,['CountryCode']_NZ,['CountryCode']_OM,['CountryCode']_PA,['CountryCode']_PE,['CountryCode']_PF,['CountryCode']_PG,['CountryCode']_PH,['CountryCode']_PK,['CountryCode']_PL,['CountryCode']_PR,['CountryCode']_PS,['CountryCode']_PT,['CountryCode']_PY,['CountryCode']_QA,['CountryCode']_RE,['CountryCode']_RO,['CountryCode']_RS,['CountryCode']_RU,['CountryCode']_RW,['CountryCode']_SA,['CountryCode']_SB,['CountryCode']_SC,['CountryCode']_SE,['CountryCode']_SG,['CountryCode']_SI,['CountryCode']_SJ,['CountryCode']_SK,['CountryCode']_SL,['CountryCode']_SM,['CountryCode']_SN,['CountryCode']_SO,['CountryCode']_SR,['CountryCode']_SS,['CountryCode']_ST,['CountryCode']_SV,['CountryCode']_SX,['CountryCode']_SZ,['CountryCode']_TC,['CountryCode']_TD,['CountryCode']_TG,['CountryCode']_TH,['CountryCode']_TJ,['CountryCode']_TL,['CountryCode']_TM,['CountryCode']_TN,['CountryCode']_TR,['CountryCode']_TT,['CountryCode']_TV,['CountryCode']_TW,['CountryCode']_TZ,['CountryCode']_UA,['CountryCode']_UG,['CountryCode']_UM,['CountryCode']_US,['CountryCode']_UY,['CountryCode']_UZ,['CountryCode']_VC,['CountryCode']_VE,['CountryCode']_VG,['Country

## Below is the prepared dataset for hackathon display

In [188]:
cols = ['TenantId','CountryCode','AllupSeats','PaidEXOSeats','PaidSPOSeats','PaidOD4BSeats','PaidTeamsSeats',
        'PaidSFBSeats','PaidOfficeSeats','AR_exchange_06','AR_sharepoint_06','AR_skype_06','AR_teams_06',
        'AR_od4b_06','AR_officelient_06']

In [189]:
df_dev_hackathon = df.loc[df.TenantId.isin(df_dev.TenantId), cols]
df_dev_hackathon.replace([np.nan, np.inf], [0, 1], inplace=True)

In [190]:
df_dev_hackathon['Industry'] = df.loc[df.TenantId.isin(df_dev.TenantId), 'TopParents_Industry']

In [191]:
df_dev_hackathon['Industry'].describe()

count              609
unique              23
top       Smart Spaces
freq               103
Name: Industry, dtype: object

In [192]:
df_dev_hackathon.shape

(1303, 16)

In [193]:
# set(df_dev_hackathon['Industry'])

In [194]:
labels = ['AR_exchange_06','AR_sharepoint_06','AR_skype_06','AR_teams_06','AR_od4b_06','AR_officelient_06']

def RFR_pred_outputs(Xtrain, Xdev, labels):
#     pred = []
#     cols_name = []
    
    for l in labels:
        ytrain = df_train.loc[:, l]
        ydev = df_dev.loc[:, l]
        l_ = l[3:-3]
        col_name = 'RFR_pred_{}'.format(l_)
        col_delta = 'RFR_delta_{}'.format(l_)
#         cols_name.append(col_name)
        
        ytrain = ytrain.to_numpy()
        ydev = ydev.to_numpy()
        
        model = RandomForestRegressor(n_estimators=5, random_state=0)
        model.fit(Xtrain, ytrain)

#         pred_train = model.predict(Xtrain)
#         print(mean_squared_error(ytrain, pred_train))

        pred_dev = model.predict(Xdev)
#         print(pred_dev.dtype)
#         print(mean_squared_error(ydev, pred_dev))
#         pred.append(pred_dev)
        df_dev_hackathon[col_name] = pred_dev
        df_dev_hackathon[col_delta] = pred_dev - df_dev_hackathon[l]
        
    
#     df_pred = pd.DataFrame(np.array(pred).T, columns=cols_name)
    
    return df_dev_hackathon


In [195]:
df_pred = RFR_pred_outputs(Xtrain, Xdev, labels)

In [196]:
df_pred.head()

,TenantId,CountryCode,AllupSeats,PaidEXOSeats,PaidSPOSeats,PaidOD4BSeats,PaidTeamsSeats,PaidSFBSeats,PaidOfficeSeats,AR_exchange_06,AR_sharepoint_06,AR_skype_06,AR_teams_06,AR_od4b_06,AR_officelient_06,Industry,RFR_pred_exchange,RFR_delta_exchange,RFR_pred_sharepoint,RFR_delta_sharepoint,RFR_pred_skype,RFR_delta_skype,RFR_pred_teams,RFR_delta_teams,RFR_pred_od4b,RFR_delta_od4b,RFR_pred_officelient,RFR_delta_officelient
5,0898B902-614C-4659-ACF2-17A89AEB08D6,US,12,11,12,12,11,11,0.0,0.954545,0.000000,0.000000,0.000000,0.020833,1.0,NaN,1.040336,0.085790,0.047722,0.047722,0.010003,0.010003,0.321672,0.321672,0.145034,0.124201,1.0,0.0
7,94A2F079-A05E-40DE-B09A-140AD249F7A8,US,42,38,38,41,38,38,0.0,1.557895,0.289474,0.055263,0.434211,0.185366,1.0,Partner Professional Services,0.627879,-0.930016,2.850000,2.560526,0.032381,-0.022882,0.200012,-0.234198,0.402934,0.217568,1.0,0.0
23,EDA29C30-2151-4CC8-8681-B1029A6FBC44,US,9,8,0,1,0,0,0.0,0.237500,1.000000,0.000000,0.000000,0.050000,1.0,Media & Entertainment,0.648333,0.410833,0.000000,-1.000000,0.000002,0.000002,0.000000,0.000000,0.000000,-0.050000,0.8,-0.2
33,D800138B-B5A4-466B-9F6C-9F0BDEFD1250,ZA,17,17,13,13,13,13,0.0,0.979412,0.003846,0.000000,0.038462,0.023077,1.0,NaN,0.350000,-0.629412,0.110000,0.106154,0.085242,0.085242,0.000000,-0.038462,0.621500,0.598423,1.0,0.0
36,710D1298-6DCE-49F2-B304-56EF2238E99A,TW,20,20,0,0,0,0,0.0,0.707500,0.000000,0.000000,0.000000,0.000000,1.0,NaN,0.469167,-0.238333,0.000010,0.000010,0.000003,0.000003,0.018337,0.018337,0.292500,0.292500,1.0,0.0


In [272]:
for acr, name in [('EXO', 'exchange'), ('SPO', 'sharepoint'), 
                  ('OD4B', 'od4b'), ('Teams', 'teams'), 
                  ('SFB', 'skype'), ('Office', 'officelient')]:
    def f(row):
        if row['Paid{}Seats'.format(acr)] <= 0:
            val = -1
        else:
            if row['RFR_delta_{}'.format(name)] > 0.1 and row['AR_{}_06'.format(name)] < 1:
                val = 1
            else:
                val = 0
        return val
    
    vals = []
    for i, row in df_pred.iterrows():
        vals.append(f(row))
        
    df_pred['Rec_{}'.format(acr)] = vals

In [273]:
df_pred.head()

,TenantId,CountryCode,AllupSeats,PaidEXOSeats,PaidSPOSeats,PaidOD4BSeats,PaidTeamsSeats,PaidSFBSeats,PaidOfficeSeats,AR_exchange_06,AR_sharepoint_06,AR_skype_06,AR_teams_06,AR_od4b_06,AR_officelient_06,Industry,RFR_pred_exchange,RFR_delta_exchange,RFR_pred_sharepoint,RFR_delta_sharepoint,RFR_pred_skype,RFR_delta_skype,RFR_pred_teams,RFR_delta_teams,RFR_pred_od4b,RFR_delta_od4b,RFR_pred_officelient,RFR_delta_officelient,Rec_EXO,Rec_SPO,Rec_OD4B,Rec_Teams,Rec_SFB,Rec_Office
5,0898B902-614C-4659-ACF2-17A89AEB08D6,US,12,11,12,12,11,11,0.0,0.954545,0.000000,0.000000,0.000000,0.020833,1.0,NaN,1.040336,0.085790,0.047722,0.047722,0.010003,0.010003,0.321672,0.321672,0.145034,0.124201,1.0,0.0,0,0,1,1,0,-1
7,94A2F079-A05E-40DE-B09A-140AD249F7A8,US,42,38,38,41,38,38,0.0,1.557895,0.289474,0.055263,0.434211,0.185366,1.0,Partner Professional Services,0.627879,-0.930016,2.850000,2.560526,0.032381,-0.022882,0.200012,-0.234198,0.402934,0.217568,1.0,0.0,0,1,1,0,0,-1
23,EDA29C30-2151-4CC8-8681-B1029A6FBC44,US,9,8,0,1,0,0,0.0,0.237500,1.000000,0.000000,0.000000,0.050000,1.0,Media & Entertainment,0.648333,0.410833,0.000000,-1.000000,0.000002,0.000002,0.000000,0.000000,0.000000,-0.050000,0.8,-0.2,1,-1,0,-1,-1,-1
33,D800138B-B5A4-466B-9F6C-9F0BDEFD1250,ZA,17,17,13,13,13,13,0.0,0.979412,0.003846,0.000000,0.038462,0.023077,1.0,NaN,0.350000,-0.629412,0.110000,0.106154,0.085242,0.085242,0.000000,-0.038462,0.621500,0.598423,1.0,0.0,0,1,1,0,0,-1
36,710D1298-6DCE-49F2-B304-56EF2238E99A,TW,20,20,0,0,0,0,0.0,0.707500,0.000000,0.000000,0.000000,0.000000,1.0,NaN,0.469167,-0.238333,0.000010,0.000010,0.000003,0.000003,0.018337,0.018337,0.292500,0.292500,1.0,0.0,0,-1,-1,-1,-1,-1


In [276]:
df_pred.to_csv('hackathon.csv', index=False)

In [259]:
def f(row):
    if row['PaidEXOSeats'] <= 0:
        val = -1
    else:
        if row['RFR_delta_exchange'] > 0.1 and row['AR_exchange_06'] < 1:
            val = 1
        else:
            val = 0
    return val

In [265]:
vals = []
for i, row in df_pred.iterrows():
    vals.append(f(row))

In [266]:
df_pred['Rec_EXO'] = vals

In [267]:
# df_pred['Rec_EXO'] = df_pred.apply(lambda x: f(x))

In [269]:
df_pred.head(20)

,TenantId,CountryCode,AllupSeats,PaidEXOSeats,PaidSPOSeats,PaidOD4BSeats,PaidTeamsSeats,PaidSFBSeats,PaidOfficeSeats,AR_exchange_06,AR_sharepoint_06,AR_skype_06,AR_teams_06,AR_od4b_06,AR_officelient_06,Industry,RFR_pred_exchange,RFR_delta_exchange,RFR_pred_sharepoint,RFR_delta_sharepoint,RFR_pred_skype,RFR_delta_skype,RFR_pred_teams,RFR_delta_teams,RFR_pred_od4b,RFR_delta_od4b,RFR_pred_officelient,RFR_delta_officelient,Rec_EXO
5,0898B902-614C-4659-ACF2-17A89AEB08D6,US,12,11,12,12,11,11,0.0,0.954545,0.000000,0.000000,0.000000,0.020833,1.000000,NaN,1.040336,0.085790,0.047722,0.047722,0.010003,0.010003,0.321672,0.321672,0.145034,0.124201,1.000,0.000000,0
7,94A2F079-A05E-40DE-B09A-140AD249F7A8,US,42,38,38,41,38,38,0.0,1.557895,0.289474,0.055263,0.434211,0.185366,1.000000,Partner Professional Services,0.627879,-0.930016,2.850000,2.560526,0.032381,-0.022882,0.200012,-0.234198,0.402934,0.217568,1.000,0.000000,0
23,EDA29C30-2151-4CC8-8681-B1029A6FBC44,US,9,8,0,1,0,0,0.0,0.237500,1.000000,0.000000,0.000000,0.050000,1.000000,Media & Entertainment,0.648333,0.410833,0.000000,-1.000000,0.000002,0.000002,0.000000,0.000000,0.000000,-0.050000,0.800,-0.200000,1
33,D800138B-B5A4-466B-9F6C-9F0BDEFD1250,ZA,17,17,13,13,13,13,0.0,0.979412,0.003846,0.000000,0.038462,0.023077,1.000000,NaN,0.350000,-0.629412,0.110000,0.106154,0.085242,0.085242,0.000000,-0.038462,0.621500,0.598423,1.000,0.000000,0
36,710D1298-6DCE-49F2-B304-56EF2238E99A,TW,20,20,0,0,0,0,0.0,0.707500,0.000000,0.000000,0.000000,0.000000,1.000000,NaN,0.469167,-0.238333,0.000010,0.000010,0.000003,0.000003,0.018337,0.018337,0.292500,0.292500,1.000,0.000000,0
41,E37D1B95-58AD-4016-9611-0940EE538CAF,RU,61,0,0,60,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000001,0.000001,0.000003,0.000003,0.010370,0.010370,1.000,0.000000,-1
43,C21069C6-0DA2-41D3-BEFD-B2793A1199B9,BR,166,82,83,84,82,82,164.0,1.154268,0.000000,0.007317,0.000000,0.017262,0.185061,Smart Spaces,0.855500,-0.298768,0.000017,0.000017,0.000008,-0.007309,0.000000,0.000000,0.008466,-0.008796,0.800,0.614939,0
45,1A538077-82EE-4B33-847A-EEFE5AA7E8CC,US,6,4,0,2,0,0,0.0,0.662500,0.000000,0.000000,0.000000,0.000000,0.000000,Smart Spaces,0.803833,0.141333,0.065010,0.065010,0.000009,0.000009,0.000000,0.000000,0.066126,0.066126,1.000,1.000000,1
59,1C808616-D916-43D3-A90F-575EF8C82B51,US,2,2,2,2,2,2,0.0,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.971215,0.846215,0.106667,0.106667,0.017500,0.017500,0.067152,0.067152,0.584048,0.584048,0.800,0.800000,1
62,4428DDF6-C766-4776-AE7A-7347A6818C9B,BE,13,13,13,13,13,13,0.0,0.946154,0.000000,0.000000,0.000000,0.030769,1.000000,Smart Spaces,0.981679,0.035525,0.000005,0.000005,0.032396,0.032396,0.004074,0.004074,0.213356,0.182587,1.000,0.000000,0


In [166]:
df_fake = df_pred.loc[:10,:]

In [114]:
col

'RF_predEXO'

In [ ]:
df_dev_hackathon['rec_exo'] = 

In [97]:
df_dev_hackathon.head()

,TenantId,CountryCode,AllupSeats,PaidEXOSeats,PaidSPOSeats,PaidOD4BSeats,PaidTeamsSeats,PaidSFBSeats,PaidOfficeSeats,AR_exchange_06,AR_sharepoint_06,AR_skype_06,AR_teams_06,AR_od4b_06,AR_officelient_06,pred_exo
5,0898B902-614C-4659-ACF2-17A89AEB08D6,US,12,11,12,12,11,11,0.0,0.954545,0.000000,0.000000,0.000000,0.020833,1.0,1.040336
7,94A2F079-A05E-40DE-B09A-140AD249F7A8,US,42,38,38,41,38,38,0.0,1.557895,0.289474,0.055263,0.434211,0.185366,1.0,0.627879
23,EDA29C30-2151-4CC8-8681-B1029A6FBC44,US,9,8,0,1,0,0,0.0,0.237500,1.000000,0.000000,0.000000,0.050000,1.0,0.648333
33,D800138B-B5A4-466B-9F6C-9F0BDEFD1250,ZA,17,17,13,13,13,13,0.0,0.979412,0.003846,0.000000,0.038462,0.023077,1.0,0.350000
36,710D1298-6DCE-49F2-B304-56EF2238E99A,TW,20,20,0,0,0,0,0.0,0.707500,0.000000,0.000000,0.000000,0.000000,1.0,0.469167
